In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('deepfake3text.csv')

In [4]:
from transformers import  XLNetTokenizer

# Load the base-cased XLNet model
model = XLNetModel.from_pretrained('xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [7]:
from sklearn.model_selection import train_test_split
text = df['Inputs']
label = df['Label']


In [9]:
X_train, X_val, y_train, y_val = train_test_split(text, label, test_size=0.25, stratify=label)

In [10]:
def tokenize_data(tokenizer, texts):
  """Tokenizes a list of text strings using the given BERT tokenizer.

  Args:
    tokenizer: A BERT tokenizer.
    texts: A list of text strings.

  Returns:
    A list of tokenized sentences, where each sentence is a list of token IDs.
  """

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True, max_length=128)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [11]:
x =tokenize_data(tokenizer, X_train)

In [12]:
x1 =tokenize_data(tokenizer, X_val)

In [13]:
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [22]:

embedding_layer = Embedding(
    input_dim=model.config.vocab_size,
    output_dim=model.config.hidden_size,
    weights=[model.get_input_embeddings().weight.detach()],
    trainable=False)

In [23]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    embedding_layer,
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)


In [25]:
model.fit(x_train, y_train, epochs=10,steps_per_epoch=100,batch_size=32)


Epoch 1/10
100/100 [==============================] - 6s 13ms/step - loss: 0.2184 - accuracy: 0.9156
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0983 - accuracy: 0.9694
Epoch 3/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0910 - accuracy: 0.9709
Epoch 4/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0893 - accuracy: 0.9722
Epoch 5/10
100/100 [==============================] - 1s 12ms/step - loss: 0.0641 - accuracy: 0.9806
Epoch 6/10
100/100 [==============================] - 1s 12ms/step - loss: 0.0595 - accuracy: 0.9784
Epoch 7/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0585 - accuracy: 0.9812
Epoch 8/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0532 - accuracy: 0.9828
Epoch 9/10
100/100 [==============================] - 1s 12ms/step - loss: 0.0625 - accuracy: 0.9797
Epoch 10/10
100/100 [==============================] - 1s 11ms/step - loss: 0.0519 - accura

In [26]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 1s 9ms/step - loss: 0.0491 - accuracy: 0.9866


[0.04909510165452957, 0.9865599870681763]

In [40]:
text=" The University of Chicago is thomen in PublicAffairs Council project. Learn more about the program.In order to become"
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)

# Make a prediction
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)


output = model.predict(prediction_input)




if output > 0.5:
    predicted_class = 'human written'
else:
    predicted_class = 'machine generated'

# Print the predicted class
print('Predicted class:', predicted_class,output)


1/1 [==============================] - 0s 16ms/step
Predicted class: machine generated [[0.00593289]]


In [43]:
model.save('xlnet_english_deepfakedetection.h5')

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 768)         24576000  
                                                                 
 lstm (LSTM)                 (None, 128)               459264    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 25,035,393
Trainable params: 459,393
Non-trainable params: 24,576,000
_________________________________________________________________


In [45]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')


In [46]:
x =tokenize_data(tokenizer, X_train)

In [47]:
x1 =tokenize_data(tokenizer, X_val)

In [48]:
import keras
import transformers
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
x_train=pad_sequences(x)
x_val=pad_sequences(x1)

In [49]:

embedding_layer = Embedding(
    input_dim=model.config.vocab_size,
    output_dim=model.config.hidden_size,
    weights=[model.get_input_embeddings().weight.detach()],
    trainable=False)

In [50]:

model = tf.keras.models.Sequential([
    embedding_layer,
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [51]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)


In [52]:
model.fit(x_train, y_train, epochs=10,steps_per_epoch=100,batch_size=32)


Epoch 1/10
100/100 [==============================] - 2s 13ms/step - loss: 0.2551 - accuracy: 0.9141
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1668 - accuracy: 0.9503
Epoch 3/10
100/100 [==============================] - 1s 12ms/step - loss: 0.1375 - accuracy: 0.9566
Epoch 4/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1642 - accuracy: 0.9484
Epoch 5/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1425 - accuracy: 0.9594
Epoch 6/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1389 - accuracy: 0.9588
Epoch 7/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1321 - accuracy: 0.9600
Epoch 8/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1190 - accuracy: 0.9663
Epoch 9/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1235 - accuracy: 0.9594
Epoch 10/10
100/100 [==============================] - 1s 11ms/step - loss: 0.1045 - accura

In [53]:
model.evaluate(x_val,y_val,steps=100)

100/100 [==============================] - 1s 8ms/step - loss: 0.0942 - accuracy: 0.9708


[0.0941619798541069, 0.9707599878311157]

In [56]:
text=" Make sure you have the latest version of PRAW installed, and the code above should work. If you encounter any issues, refer to the PRAW documentation or the specific version's documentation for guidance."
text_p = [text]
x2 = tokenize_data(tokenizer, text_p)

# Make a prediction
prediction_input = np.array(x2).reshape(-1)
prediction_input=pad_sequences([prediction_input],128)


output = model.predict(prediction_input)




if output > 0.5:
    predicted_class = 'human written'
else:
    predicted_class = 'machine generated'

# Print the predicted class
print('Predicted class:', predicted_class,output)


1/1 [==============================] - 0s 16ms/step
Predicted class: machine generated [[0.00894152]]


In [57]:
model.save('distbertdeepfakedetection.h5')

In [58]:
import re

def is_arabic(text):
    arabic_regex = re.compile(r'[\u0600-\u06FF]+')
    return arabic_regex.search(text) != None

def is_english(text):
    english_regex = re.compile(r'[a-zA-Z]+')
    return english_regex.search(text) != None

def check_language(text):
    if is_arabic(text):
        print("The text is Arabic.")
    elif is_english(text):
        print("The text is English.")
    else:
        print("The text is neither Arabic nor English.")


In [61]:
text='مرحبا بالعالم!'

In [62]:
check_language(text)

The text is Arabic.
